In [35]:
from PIL import Image
import pytesseract
# Install required libraries
import requests
import json
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

In [43]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Load the image from file
image_path = 'test4.jpeg'  # Replace with your image file path
image = Image.open(image_path)

In [44]:
extracted_text = pytesseract.image_to_string(image)

# Print the extracted text
print("Extracted Text:")
print(extracted_text)

Extracted Text:
Liquor Street
(ODVJH Private Limited)
11/2 Sector- 37,
Faridabad- 121003.
Ph. No. : 0129-4360377, 9311111116
GSTIN : O6@AACCO6344G12J

Invoice Number: IN001001259
Invoice Date: 20-May-18 22:55

item Qty. Rate Total
Tandoori chicken 4 295.00 309.75
Lasooni Dal Tadka 4 275.00 288.75
HYDERABADI MURG)
BIRYANI 1 375.00 393.75
Tandoori Roti all food
less spicy 2 30.00 63.00
Tandoori Roti 1 30.00 31.50
Total Qty: | 6
Sub Total: 1,035.00
CGST@z2.5 25.89
SGST@2.5 25.89
Total: 1,139.00

Thanks For Visit...



In [45]:
def preprocess_text(text):
    # Tokenize into sentences
    sentences = sent_tokenize(text)
    print("\nTokenized Sentences:")
    print(sentences)
    
    # Tokenize into words
    words = word_tokenize(text)
    print("\nTokenized Words:")
    print(words)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    print("\nFiltered Words (Stopwords Removed):")
    print(filtered_words)

    # Calculate word frequency
    freq_dist = FreqDist(filtered_words)
    print("\nWord Frequencies:")
    for word, freq in freq_dist.most_common(10):
        print(f"{word}: {freq}")

    return {
        "sentences": sentences,
        "filtered_words": filtered_words,
        "freq_dist": freq_dist
    }



In [46]:
# Process the extracted text
processed_data = preprocess_text(extracted_text)
print(processed_data)


Tokenized Sentences:
['Liquor Street\n(ODVJH Private Limited)\n11/2 Sector- 37,\nFaridabad- 121003.', 'Ph.', 'No.', ': 0129-4360377, 9311111116\nGSTIN : O6@AACCO6344G12J\n\nInvoice Number: IN001001259\nInvoice Date: 20-May-18 22:55\n\nitem Qty.', 'Rate Total\nTandoori chicken 4 295.00 309.75\nLasooni Dal Tadka 4 275.00 288.75\nHYDERABADI MURG)\nBIRYANI 1 375.00 393.75\nTandoori Roti all food\nless spicy 2 30.00 63.00\nTandoori Roti 1 30.00 31.50\nTotal Qty: | 6\nSub Total: 1,035.00\nCGST@z2.5 25.89\nSGST@2.5 25.89\nTotal: 1,139.00\n\nThanks For Visit...']

Tokenized Words:
['Liquor', 'Street', '(', 'ODVJH', 'Private', 'Limited', ')', '11/2', 'Sector-', '37', ',', 'Faridabad-', '121003', '.', 'Ph', '.', 'No', '.', ':', '0129-4360377', ',', '9311111116', 'GSTIN', ':', 'O6', '@', 'AACCO6344G12J', 'Invoice', 'Number', ':', 'IN001001259', 'Invoice', 'Date', ':', '20-May-18', '22:55', 'item', 'Qty', '.', 'Rate', 'Total', 'Tandoori', 'chicken', '4', '295.00', '309.75', 'Lasooni', 'Dal', 'Tad

In [49]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

def summarize_with_gemini(api_key, extracted_text, custom_prompt):
    """
    Summarize text using Google's Gemini API
    """
    # Define the API endpoint - using the correct Gemini API URL
    endpoint = f"https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={api_key}"
    
    # Construct the payload according to Gemini API specifications
    payload = {
        "contents": [{
            "parts": [{
                "text": f"Instructions: {custom_prompt}\n\nText to analyze: {extracted_text}"
            }]
        }],
        "generationConfig": {
            "temperature": 0.7,
            "maxOutputTokens": 300,
            "topP": 0.8,
            "topK": 40
        }
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    try:
        # Send the POST request
        response = requests.post(endpoint, headers=headers, json=payload)
        
        # Check if the request was successful
        if response.status_code == 200:
            response_data = response.json()
            
            # Extract the generated text from the response
            if ('candidates' in response_data and 
                len(response_data['candidates']) > 0 and 
                'content' in response_data['candidates'][0] and 
                'parts' in response_data['candidates'][0]['content'] and 
                len(response_data['candidates'][0]['content']['parts']) > 0 and 
                'text' in response_data['candidates'][0]['content']['parts'][0]):
                
                return {
                    "success": True,
                    "summary": response_data['candidates'][0]['content']['parts'][0]['text']
                }
            else:
                return {
                    "success": False,
                    "error": "Unexpected response structure"
                }
        else:
            error_message = response.json().get('error', {}).get('message', 'Unknown error occurred')
            return {
                "success": False,
                "error": f"API request failed with status {response.status_code}: {error_message}"
            }
            
    except requests.exceptions.RequestException as e:
        return {
            "success": False,
            "error": f"Request failed: {str(e)}"
        }

# Example usage
if __name__ == "__main__":
    # Your API key
    api_key = os.getenv('GEMINI_API_KEY') # Replace with actual API key
    
    # Example text and prompt
    extracted_text = pytesseract.image_to_string(image)
    custom_prompt = "explain all the dishes?"
    
    # Call the function
    result = summarize_with_gemini(api_key, extracted_text, custom_prompt)
    
    # Handle the result
    if result.get("success", False):
        print("Summary:", result["summary"])
    else:
        print("Error:", result.get("error", "An unknown error occurred"))

Summary: **Tandoori chicken:** A classic Indian dish made with chicken marinated in yogurt and spices, then grilled in a tandoor (clay oven).

**Lasooni Dal Tadka:** A lentil dish made with black lentils, garlic, and spices.

**HYDERABADI MURG BIRYANI:** A biryani dish made with chicken, rice, and spices.

**Tandoori Roti all food less spicy:** A type of Indian flatbread made with whole wheat flour and cooked in a tandoor. This version is less spicy than the traditional tandoori roti.

**Tandoori Roti:** A type of Indian flatbread made with whole wheat flour and cooked in a tandoor.
